In [32]:
import urllib
import pandas as pd
import numpy as np
import datetime
import string
import pymysql as mdb

In [33]:
topicsfile="/Users/abramvandergeest/Dropbox/insight_work/topic_list.txt"
f=open(topicsfile)
topics=f.readlines()
f.close()

In [34]:
con = mdb.connect('localhost', 'abram.ghost', '', 'wikidata')
####I NEED TO SET THIS TO A NON-GITHUB AUTHENTICATION LOCATION AND CHANGE PASSWORD

#I should also create the  database if it doesn't exist

In [35]:
#Makes topics table
with con:
    cur = con.cursor()
    cur.execute("DROP TABLE IF EXISTS `topics`;")
    str="CREATE TABLE `topics` (`Id` INT PRIMARY KEY AUTO_INCREMENT, `topic_label` VARCHAR(64), `topic_string` VARCHAR(64));"
    cur.execute(str)
    

In [36]:
#Populate topics table
with con:
    for topic in topics:
        label="".join(topic.lower().split())
        string=topic.rstrip()
        sql="INSERT INTO `topics` (`topic_label`,`topic_string`) VALUES (%s,%s);"
        cur.execute(sql,(label,string))

In [37]:
#Makes page_views table
with con:
    cur = con.cursor()
    cur.execute("DROP TABLE IF EXISTS `page_views`;")
    str="CREATE TABLE `page_views` (`Id` INT PRIMARY KEY AUTO_INCREMENT, `topic_id` INT, `count` INT, `date` VARCHAR(64));"
    cur.execute(str)
    

In [28]:
#creating a dataframe with all english pages of a large size
fn="/Users/abramvandergeest/wikidata/pagecounts-20150501-000000.gz"
fo="/Users/abramvandergeest/wikidata/temp.gz"
fo2="/Users/abramvandergeest/wikidata/temp.txt"
urllib.urlretrieve (fn,fo)
!gunzip -c $fo>$fo2
allwiki=pd.read_csv(fo2,delim_whitespace=True)
allwiki.columns=['lang','article','views','size']
enwiki=allwiki[allwiki['lang']=='en']
enwiki=enwiki[enwiki['size']>=50000]

In [48]:
#
date='2015050100'
dic={}
views=[]
with con:
    cur = con.cursor()
    sql="SELECT `Id`,`topic_label`,`topic_string` FROM `topics`;"
    cur.execute(sql)
    for row in cur:
        tops=row[2].rstrip().split()
        twiki=enwiki[np.logical_and.reduce([enwiki['article'].str.contains(top,case=False) for top in tops ])==True]
        t=row[2].rstrip()
        dic[t]=twiki['views'].sum(axis=0)
        #print row[0],dic[t],date
        views.append([row[0],dic[t],date])
        

for view in views:
    print view

In [49]:
with con:
    cur = con.cursor()
    for view in views:
        sql="INSERT INTO `page_views` (`topic_id`,`count`,`date`) VALUES (%s,%s,%s);"
        cur.execute(sql,view)